In [1]:
import json
from mesa import Agent, Model
from mesa.time import RandomActivation, SimultaneousActivation
import numpy as np
import geopandas as gpd
import contextily as ctx
from mesa.datacollection import DataCollector
import skmob
import time
from mesa.batchrunner import BatchRunnerMP
import networkx as nx
import pickle
from tqdm import tqdm
import osmnx as ox

# Read jsons

In [2]:
with open('population_dist.json') as f:
  population_dist = json.load(f)

In [3]:
with open('regions_num_dist.json') as f:
  regions_num_dist = json.load(f)

In [4]:
with open('travel_dist.json') as f:
  travel_dist = json.load(f)

In [5]:
with open('neighbors_dist.json') as f:
  neighbors_dist = json.load(f)

In [6]:
with open('region_nodes.json') as f:
  region_nodes = json.load(f)

In [7]:
with open('neighbors_dict.json') as f:
  neighbors_dict = json.load(f)

# Process distributions

In [8]:
s = np.array(list(population_dist.values())).astype(int).sum()
for key, value in population_dist.items():
    population_dist[key] = int(population_dist[key]) / s

In [9]:
for key, value in travel_dist.items():
    s = travel_dist[key]['true'] + travel_dist[key]['false']
    travel_dist[key]['true'] = travel_dist[key]['true'] / s
    travel_dist[key]['false'] = 1 - travel_dist[key]['true']

In [10]:
for key, value in regions_num_dist.items():
    s = np.array(list(value.values())).astype(int).sum()
    if s != 0:
        for i in range(1, 34):
            regions_num_dist[key][str(i)] = regions_num_dist[key][str(i)] / s

In [11]:
for key, value in neighbors_dist.items():
    s = np.array(list(value.values())).astype(int).sum()
    if s != 0:
        for k, v in value.items():
            neighbors_dist[key][k] = neighbors_dist[key][k] / s

In [12]:
# neighbors_dist

# Regions

In [13]:
regions = gpd.read_file('../../data/raw/EtapII-REJONY_wroclaw.shp')
regions = regions.to_crs(epsg=3857)
# regions.head(3)

In [14]:
# regions.info()

# Graph

In [15]:
with open('roads_graph.pickle', 'rb') as f:
    road_graph = pickle.load(f)
road_graph = road_graph.to_undirected()

In [16]:
# region_nodes = {}
# for i in range(500):
#     region_nodes[i] = np.random.choice(list(road_graph.nodes()), 10)

# MESA

In [17]:
class Person(Agent):

    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)

        self.start_region = self._sample_region()
        self.current_region = self.start_region
        self.path = [self._sample_node(self.start_region)]
        self.on_the_way = self._sample_status()
        self.end_region = self.start_region
        
        if self.on_the_way:
            self.regions_num_left = self._sample_regions_num() - 1
        else:
            self.regions_num_left = 0

        self.regions_num = self.regions_num_left


    def _sample_region(self):

        sample = np.random.multinomial(1, list(population_dist.values()))
        sample = np.argmax(sample)
        region = list(population_dist.keys())[sample]

        return region

    def _sample_status(self):

        try:
            sample = np.random.multinomial(1, list(travel_dist[str(self.start_region)].values()))
        except KeyError:
            sample = np.random.multinomial(1, np.zeros(2))
        status = not np.argmax(sample)

        return status

    def _sample_regions_num(self):

        try:
            sample = np.random.multinomial(1, list(regions_num_dist[str(self.start_region)].values()))
        except KeyError:
            sample = np.random.multinomial(1, np.zeros(33))
        regions_num = np.argmax(sample) + 1

        return regions_num

    def _sample_node(self, region):
        
        nodes_in_region = region_nodes[str(region)]
        if len(nodes_in_region) > 0:
            node = np.random.choice(nodes_in_region)
        else:
            nodes_in_region = []
            for n in neighbors_dict[str(region)]:
                nodes_in_region = nodes_in_region + region_nodes[str(n)]
            node = np.random.choice(nodes_in_region)

        return node

    def move(self):

        self.current_region = self._sample_next_region()
        self.regions_num_left = self.regions_num_left - 1

        next_region_node = self._sample_node(self.current_region)
        path = nx.shortest_path(road_graph, self.path[-1], next_region_node)
        self.path = self.path + path[1:]

    def _sample_next_region(self):

        sample = np.random.multinomial(1, list(neighbors_dist[str(self.current_region)].values()))
        sample = np.argmax(sample)
        next_region = int(list(neighbors_dist[str(self.current_region)].keys())[sample])

        return next_region

    def step(self):
        
        if self.on_the_way:

            self.move()            

            if self.regions_num_left == 0:
                self.on_the_way = False
                self.end_region = self.current_region

In [18]:
class TrafficModel(Model):
    """A model with some number of agents."""
    def __init__(self, N):
        self.num_agents = N
        self.schedule = RandomActivation(self)
        self.running = True
        # Create agents
        for i in range(self.num_agents):
            a = Person(i, self)
            self.schedule.add(a)

    def step(self):
        self.schedule.step()

In [19]:
fixed_params = {}

variable_params = {"N": [300]}

batch_run = BatchRunnerMP(
    TrafficModel,
    nr_processes=7,
    variable_parameters=variable_params,
    fixed_parameters=fixed_params,
    iterations=7,
    max_steps=34,
    agent_reporters={
        "regions_num": "regions_num",
        "start_region": "start_region",
        "end_region": "end_region",
        "path": "path"
    }
)

start_time = time.time()
batch_run.run_all()
print()
print(round(time.time() - start_time), 's')

7it [00:00, 11.36it/s]
1 s



In [20]:
results = batch_run.get_agent_vars_dataframe().reset_index()

results_flow = results[['start_region', 'end_region', 'AgentId']].groupby(['start_region', 'end_region']).count().reset_index()
results_flow['start_region'] = results_flow['start_region'].astype(int)
results_flow['end_region'] = results_flow['end_region'].astype(int)
results_flow['AgentId'] = results_flow['AgentId'].astype(int)

results_paths = results['path'].to_list()

In [21]:
# nx.set_edge_attributes(road_graph, 0, 'flow')
# for path in tqdm(results_paths):
#     for i in range(len(path) - 1):
#         try:
#             prev_flow = nx.get_edge_attributes(road_graph, 'flow')[(path[i], path[i+1], 0)]
#         except KeyError:
#             prev_flow = nx.get_edge_attributes(road_graph, 'flow')[(path[i+1], path[i], 0)]
#         nx.set_edge_attributes(road_graph, {(path[i], path[i+1], 0): {'flow': prev_flow + 1}})

In [22]:
regions = regions.to_crs(epsg=4326)

In [23]:
fdf = skmob.FlowDataFrame(
    data=results_flow,
    origin='start_region',
    destination='end_region',
    flow='AgentId',
    tile_id='NUMBER',
    tessellation=regions
)

In [24]:
m = fdf.plot_tessellation(tiles='OpenStreetMap')
# fdf.plot_flows(m, tiles='OpenStreetMap', min_flow=200, flow_weight=2)

In [25]:
for path in tqdm(np.random.choice(results_paths, 500)):
    if len(path) > 1:
        ox.folium.plot_graph_folium(road_graph.subgraph(path), graph_map=m, weight=6, opacity=0.05)

100%|██████████| 500/500 [00:19<00:00, 25.78it/s]


In [1]:
# m